In [85]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [86]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [87]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [88]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
#BASE URL
BaseURL = 'https://api-web.nhle.com'

In [97]:
STARTINGCSV2023=pd.read_csv('LastYear2023Stats.csv')  
STARTINGCSV2024=pd.read_csv('ThisYear2024Stats.csv')  

STARTINGCSV=pd.concat([STARTINGCSV2023, STARTINGCSV2024])

STARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,points,plusMinus,powerPlayGoals,powerPlayPoints,gameWinningGoals,otGoals,shots,shifts,shorthandedPoints,shorthandedGoals,opponentAbbrev,pim,toi,playerId,firstName,lastName
10123,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,0,0,0,0,0,0,17,0,0,NSH,0,13:41,8475177,Calvin,de Haan
10676,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,-1,0,0,0,0,1,18,0,0,NSH,2,11:42,8480246,Nick,Perbix
4160,2023020003,VGK,H,2023-10-10,1,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,1,0,0,0,0,3,23,0,0,SEA,0,19:53,8478403,Jack,Eichel
8731,2023020001,NSH,R,2023-10-10,1,0,{'default': 'Predators'},{'default': 'Lightning'},1,1,0,0,0,0,2,20,0,0,TBL,0,13:43,8481704,Juuso,Parssinen
10216,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},0,0,0,0,0,0,1,23,0,0,NSH,0,14:01,8476822,Luke,Glendening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2023020622,PHI,H,2024-01-08,0,0,{'default': 'Flyers'},{'default': 'Penguins'},0,0,0,0,0,0,2,26,0,0,PIT,2,18:21,8476372,Nick,Seeler
752,2023020622,PHI,H,2024-01-08,0,0,{'default': 'Flyers'},{'default': 'Penguins'},0,-1,0,0,0,0,2,13,0,0,PIT,0,7:43,8475235,Nicolas,Deslauriers
749,2023020622,PHI,H,2024-01-08,0,0,{'default': 'Flyers'},{'default': 'Penguins'},0,-2,0,0,0,0,2,20,0,0,PIT,2,14:20,8474715,Cam,Atkinson
779,2023020622,PHI,H,2024-01-08,0,0,{'default': 'Flyers'},{'default': 'Penguins'},0,0,0,0,0,0,6,22,0,0,PIT,0,22:16,8478439,Travis,Konecny


In [90]:
#TEAMS
import requests

link = BaseURL+'/v1/standings/now'
f = requests.get(link)

i=0
teamlist=[]
abbrevlist=[]
citylist=[]

while ((len(teamlist)<32)|(len(teamlist)<32)):
    try:
        citylist.append(f.json()['standings'][i]['placeName']['default'])
        teamlist.append(f.json()['standings'][i]['teamCommonName']['default'])
        abbrevlist.append(f.json()['standings'][i]['teamAbbrev']['default'])
        #teamlist=list(set(teamlist))
        i=i+1
    except:
        break
len(teamlist)

Teams=pd.DataFrame()
Teams['City']=citylist
Teams['TeamName']=teamlist
Teams['TeamID']=abbrevlist
Teams

,City,TeamName,TeamID
0,Winnipeg,Jets,WPG
1,Boston,Bruins,BOS
2,Vancouver,Canucks,VAN
3,Colorado,Avalanche,COL
4,NY Rangers,Rangers,NYR
5,Florida,Panthers,FLA
6,Dallas,Stars,DAL
7,Vegas,Golden Knights,VGK
8,Carolina,Hurricanes,CAR
9,Toronto,Maple Leafs,TOR


In [91]:
playerStats=STARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','shots','opponentAbbrev']]

playerStats['GameNumber'] = playerStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
playerStats['GP']=playerStats.groupby('teamAbbrev').GameNumber.transform(np.max)

playerStats['SeasonAvgSOG']= playerStats.groupby(['playerId']).shots.transform(np.sum)/playerStats['GP']
playerStats['OverallZScoreShots'] = np.abs(stats.zscore(playerStats['shots']))

playerStats['LastFiveFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-5, 1, 0)

playerStats['LastFiveAvgShots']=playerStats.groupby(['playerId','LastFiveFlag']).shots.transform(np.sum)/5
playerStats['LastFiveAvgShots']=np.where(playerStats['LastFiveFlag'] == 1, playerStats['LastFiveAvgShots'], 0)
playerStats['LastFiveAvgShots']=playerStats.groupby('playerId').LastFiveAvgShots.transform(np.max)

playerStats['LastTenFlag'] = np.where(playerStats['GameNumber'] >= playerStats['GP']-10, 1, 0)

playerStats['LastTenAvgShots']=playerStats.groupby(['playerId','LastTenFlag']).shots.transform(np.sum)/10
playerStats['LastTenAvgShots']=np.where(playerStats['LastTenFlag'] == 1, playerStats['LastTenAvgShots'], 0)
playerStats['LastTenAvgShots']=playerStats.groupby('playerId').LastTenAvgShots.transform(np.max)


playerStats

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,39,39,0.923077,0.464383,1,0.6,1,0.9
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,38,39,0.923077,0.464383,1,0.6,1,0.9
2,WPG,8475455,Brenden,Dillon,2023-12-27,0,CHI,37,39,0.923077,1.105210,1,0.6,1,0.9
3,WPG,8475455,Brenden,Dillon,2023-12-22,0,BOS,36,39,0.923077,1.105210,1,0.6,1,0.9
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,35,39,0.923077,0.464383,1,0.6,1,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,SJS,8482222,Alexander,Barabanov,2024-01-04,3,WPG,2,40,0.700000,0.817273,0,0.0,0,0.0
1844,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,40,0.700000,0.464383,0,0.0,0,0.0
1845,SJS,8482667,William,Eklund,2024-01-06,0,TOR,3,40,1.500000,1.105210,0,2.6,0,2.3
1846,SJS,8482667,William,Eklund,2024-01-04,0,WPG,2,40,1.500000,1.105210,0,2.6,0,2.3


In [92]:
res = playerStats.groupby(['playerId'])["shots"].quantile([0.05, 0.95]).unstack(level=1)
res.reset_index(inplace=True)
res=res.set_axis(['playerId','Low','High'], axis=1)
res=playerStats.merge(res,on=['playerId'],how='left')
ShotsNoOutliers=res[(res['Low']<res['shots'])&(res['shots']<res['High'])].drop(columns=['Low', 'High'])

ShotsNoOutliers['GameNumberShotsNoOut'] = ShotsNoOutliers.groupby(['playerId'])['shots'].cumcount(ascending=False)+1
ShotsNoOutliers['GPShotsNoOut']=ShotsNoOutliers.groupby(['playerId']).GameNumberShotsNoOut.transform(np.max)
ShotsNoOutliers['NoOutliersAvgSOG']= ShotsNoOutliers.groupby(['playerId']).shots.transform(np.sum)/ShotsNoOutliers['GPShotsNoOut']

ShotsNoOutliers

,teamAbbrev,playerId,firstName,lastName,gameDate,shots,opponentAbbrev,GameNumber,GP,SeasonAvgSOG,OverallZScoreShots,LastFiveFlag,LastFiveAvgShots,LastTenFlag,LastTenAvgShots,GameNumberShotsNoOut,GPShotsNoOut,NoOutliersAvgSOG
0,WPG,8475455,Brenden,Dillon,2023-12-31,1,MIN,39,39,0.923077,0.464383,1,0.6,1,0.9,21,21,1.285714
1,WPG,8475455,Brenden,Dillon,2023-12-30,1,MIN,38,39,0.923077,0.464383,1,0.6,1,0.9,20,21,1.285714
4,WPG,8475455,Brenden,Dillon,2023-12-20,1,DET,35,39,0.923077,0.464383,1,0.6,1,0.9,19,21,1.285714
6,WPG,8475455,Brenden,Dillon,2023-12-16,1,COL,33,39,0.923077,0.464383,0,0.6,1,0.9,18,21,1.285714
7,WPG,8475455,Brenden,Dillon,2023-12-13,2,LAK,32,39,0.923077,0.176445,0,0.6,1,0.9,17,21,1.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21844,SJS,8481567,Henry,Thrun,2024-01-06,1,TOR,3,40,0.525000,0.464383,0,0.0,0,0.0,2,11,1.636364
21846,SJS,8481567,Henry,Thrun,2024-01-02,2,DET,1,40,0.525000,0.176445,0,0.0,0,0.0,1,11,1.636364
21848,SJS,8482222,Alexander,Barabanov,2024-01-04,3,WPG,2,40,0.700000,0.817273,0,0.0,0,0.0,2,12,1.666667
21849,SJS,8482222,Alexander,Barabanov,2024-01-02,1,DET,1,40,0.700000,0.464383,0,0.0,0,0.0,1,12,1.666667


In [93]:
Teams

,City,TeamName,TeamID
0,Winnipeg,Jets,WPG
1,Boston,Bruins,BOS
2,Vancouver,Canucks,VAN
3,Colorado,Avalanche,COL
4,NY Rangers,Rangers,NYR
5,Florida,Panthers,FLA
6,Dallas,Stars,DAL
7,Vegas,Golden Knights,VGK
8,Carolina,Hurricanes,CAR
9,Toronto,Maple Leafs,TOR


In [94]:
import datetime
todaysindex=datetime.datetime.today().weekday()+2
if  todaysindex>6:
    todaysindex-=7

TodaysGames=pd.DataFrame()
link = BaseURL + '/v1/scoreboard/now' 
f = requests.get(link)
df=pd.DataFrame(f.json()['gamesByDate'][todaysindex]['games'])

for i in range(0,len(df)):
    link = BaseURL + '/v1/scoreboard/now' 
    f = requests.get(link)
    AwayTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['awayTeam']['abbrev']
    HomeTeam=f.json()['gamesByDate'][todaysindex]['games'][i]['homeTeam']['abbrev']

    dict = {'AwayTeam':[AwayTeam], 
        'HomeTeam':[HomeTeam]} 

    ThisGame=pd.DataFrame(dict)
    TodaysGames=pd.concat([TodaysGames,ThisGame], ignore_index = True)

TodaysGames


,AwayTeam,HomeTeam
0,SEA,BUF
1,LAK,TBL
2,SJS,TOR
3,VAN,NYI
4,ANA,NSH
5,FLA,STL
6,CBJ,WPG
7,EDM,CHI
8,BOS,ARI
9,OTT,CGY


In [133]:
link = BaseURL + '/v1/club-stats/TOR/now'
f = requests.get(link)
goalies=pd.DataFrame(f.json()['goalies'])
goalies

,playerId,headshot,firstName,lastName,gamesPlayed,gamesStarted,wins,losses,ties,overtimeLosses,goalsAgainstAverage,savePercentage,shotsAgainst,saves,goalsAgainst,shutouts,goals,assists,points,penaltyMinutes,timeOnIce
0,8474889,https://assets.nhle.com/mugs/nhl/20232024/TOR/...,{'default': 'Martin'},{'default': 'Jones'},11,9,7,3,0,0,2.079422,0.932432,296,276,20,2,0,0,0,0,34625
1,8479361,https://assets.nhle.com/mugs/nhl/20232024/TOR/...,{'default': 'Joseph'},{'default': 'Woll'},15,13,8,5,0,1,2.795825,0.916327,490,449,41,0,0,0,0,0,52793


In [134]:
link = BaseURL + '/v1/player/'+'8474889'+'/game-log/20232024/2'
f = requests.get(link)
df=pd.DataFrame(f.json()['gameLog'])
df

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,gamesStarted,decision,shotsAgainst,goalsAgainst,savePctg,shutouts,opponentAbbrev,pim,toi
0,2023020613,TOR,R,2024-01-06,0,0,{'default': 'Maple Leafs'},{'default': 'Sharks'},1,W,24,1,0.958333,0,SJS,0,59:56
1,2023020588,TOR,R,2024-01-03,0,0,{'default': 'Maple Leafs'},{'default': 'Ducks'},1,W,28,1,0.964286,0,ANA,0,61:30
2,2023020585,TOR,R,2024-01-02,0,0,{'default': 'Maple Leafs'},{'default': 'Kings'},1,W,31,0,1.000000,1,LAK,0,60:00
3,2023020561,TOR,H,2023-12-30,0,0,{'default': 'Maple Leafs'},{'default': 'Hurricanes'},1,L,27,2,0.925926,0,CAR,0,58:54
4,2023020531,TOR,H,2023-12-27,0,0,{'default': 'Maple Leafs'},"{'default': 'Senators', 'fr': 'Sénateurs'}",1,L,29,3,0.896552,0,OTT,0,56:57
5,2023020517,TOR,R,2023-12-23,0,0,{'default': 'Maple Leafs'},{'default': 'Blue Jackets'},1,W,28,1,0.964286,0,CBJ,0,60:00
6,2023020498,TOR,R,2023-12-21,0,0,{'default': 'Maple Leafs'},{'default': 'Sabres'},0,NaN,15,4,0.733333,0,BUF,0,31:38
7,2023020489,TOR,H,2023-12-19,0,0,{'default': 'Maple Leafs'},{'default': 'Rangers'},1,L,35,4,0.885714,0,NYR,0,58:47
8,2023020466,TOR,H,2023-12-16,0,0,{'default': 'Maple Leafs'},{'default': 'Penguins'},1,W,38,0,1.000000,1,PIT,0,60:00
9,2023020432,TOR,R,2023-12-12,0,0,{'default': 'Maple Leafs'},{'default': 'Rangers'},1,W,31,3,0.903226,0,NYR,0,59:45


In [113]:
shotSummary=ShotsNoOutliers[['teamAbbrev',	'playerId',	'firstName', 'lastName', 'SeasonAvgSOG', 'LastFiveAvgShots','LastTenAvgShots', 'NoOutliersAvgSOG']].drop_duplicates()

teamsPlayingToday=TodaysGames['AwayTeam'].tolist()+TodaysGames['HomeTeam'].tolist()
allgames=pd.concat([TodaysGames,TodaysGames[['HomeTeam','AwayTeam']].rename(
    columns={"AwayTeam": "HomeTeam","HomeTeam": "AwayTeam"})], ignore_index = True)

todaysSkaters=shotSummary[shotSummary['teamAbbrev'].isin(teamsPlayingToday)]

todaysSkaters=todaysSkaters.merge(allgames.rename(columns={"AwayTeam": "Opponent"})
                                  , left_on='teamAbbrev', right_on='HomeTeam', how='left').drop(columns=['HomeTeam'])

todaysSkaters.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(20)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,Opponent
0,BOS,8477956,David,Pastrnak,4.948718,5.8,6.2,5.193548,ARI
1,TBL,8476453,Nikita,Kucherov,4.341463,3.8,3.6,4.838710,LAK
2,TOR,8477939,William,Nylander,4.405405,4.0,4.1,4.714286,SJS
3,TOR,8479318,Auston,Matthews,4.351351,3.6,5.0,4.322581,SJS
4,OTT,8480801,Brady,Tkachuk,4.542857,5.4,5.0,4.310345,CGY
5,NSH,8476887,Filip,Forsberg,3.850000,5.8,4.1,4.096774,ANA
6,WPG,8478398,Kyle,Connor,2.615385,0.0,0.0,3.900000,CBJ
7,BUF,8475784,Jeff,Skinner,3.275000,2.2,2.6,3.653846,SEA
8,FLA,8477409,Carter,Verhaeghe,3.435897,3.6,3.8,3.625000,STL
9,TOR,8475166,John,Tavares,3.783784,4.8,5.1,3.548387,SJS


In [156]:
goalieSTARTINGCSV2023=pd.read_csv('GoalieLastYear2023Stats.csv')  
goalieSTARTINGCSV2024=pd.read_csv('GoalieThisYear2024Stats.csv')  

goalieSTARTINGCSV=pd.concat([goalieSTARTINGCSV2023, goalieSTARTINGCSV2024])
goalieSTARTINGCSV=goalieSTARTINGCSV[goalieSTARTINGCSV['gamesStarted']==1]
goalieSTARTINGCSV.sort_values(by=['gameDate'])

,gameId,teamAbbrev,homeRoadFlag,gameDate,goals,assists,commonName,opponentCommonName,gamesStarted,decision,shotsAgainst,goalsAgainst,savePctg,shutouts,opponentAbbrev,pim,toi,playerId,firstName,lastName
1098,2023020002,CHI,R,2023-10-10,0,0,{'default': 'Blackhawks'},{'default': 'Penguins'},1,W,41,2,0.951220,0,PIT,0,60:00,8475852,Petr,Mrazek
760,2023020003,SEA,R,2023-10-10,0,0,{'default': 'Kraken'},{'default': 'Golden Knights'},1,L,27,3,0.888889,0,VGK,0,59:37,8475831,Philipp,Grubauer
270,2023020003,VGK,H,2023-10-10,0,0,{'default': 'Golden Knights'},{'default': 'Kraken'},1,W,33,1,0.969697,0,SEA,0,60:00,8478499,Adin,Hill
672,2023020001,TBL,H,2023-10-10,0,0,{'default': 'Lightning'},{'default': 'Predators'},1,W,31,3,0.903226,0,NSH,0,60:00,8477992,Jonas,Johansson
586,2023020002,PIT,H,2023-10-10,0,0,{'default': 'Penguins'},{'default': 'Blackhawks'},1,L,35,3,0.914286,0,CHI,0,57:30,8477465,Tristan,Jarry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,2023020624,COL,H,2024-01-08,0,0,{'default': 'Avalanche'},{'default': 'Bruins'},1,W,26,3,0.884615,0,BOS,0,65:00,8480382,Alexandar,Georgiev
84,2023020623,MIN,H,2024-01-08,0,0,{'default': 'Wild'},{'default': 'Stars'},1,L,20,4,0.800000,0,DAL,0,59:58,8470594,Marc-Andre,Fleury
59,2023020622,PIT,R,2024-01-08,0,0,{'default': 'Penguins'},{'default': 'Flyers'},1,W,37,1,0.972973,0,PHI,0,60:00,8477968,Alex,Nedeljkovic
6,2023020624,BOS,R,2024-01-08,0,0,{'default': 'Bruins'},{'default': 'Avalanche'},1,O,36,3,0.916667,0,COL,0,65:00,8480280,Jeremy,Swayman


In [165]:
goalieStats=goalieSTARTINGCSV[['teamAbbrev','playerId','firstName','lastName','gameDate','opponentAbbrev','gamesStarted','shotsAgainst','goalsAgainst','savePctg']]

goalieStats['GameNumber'] = goalieStats.groupby(['playerId'])['gameDate'].cumcount(ascending=False)+1
goalieStats['GP']=goalieStats.groupby(['playerId']).gamesStarted.transform(np.sum)

goalieStats['AvgShotsAgainst']=goalieStats.groupby(['playerId']).shotsAgainst.transform(np.sum)/goalieStats['GP']
goalieStats['RankAvgShotsAgainst']=goalieStats['AvgShotsAgainst'].rank(pct=True)*100

goalieStats['AvgGoalsAgainst']=goalieStats.groupby(['playerId']).goalsAgainst.transform(np.sum)/goalieStats['GP']
goalieStats['RankAvgGoalsAgainst']=goalieStats['AvgGoalsAgainst'].rank(pct=True)*100

goalieStats['AvgSavePctg']=goalieStats.groupby(['playerId']).savePctg.transform(np.sum)/goalieStats['GP']
goalieStats['RankAvgSavePctg']=goalieStats['AvgSavePctg'].rank(pct=True)*100

goalieStats['LastFiveFlag'] = np.where(goalieStats['GameNumber'] >= goalieStats['GP']-5, 1, 0)

goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby(['playerId','LastFiveFlag']).shotsAgainst.transform(np.sum)/5
goalieStats['LastFiveAvgShotsAgainst']=np.where(goalieStats['LastFiveFlag'] == 1, goalieStats['LastFiveAvgShotsAgainst'], 0)
goalieStats['LastFiveAvgShotsAgainst']=goalieStats.groupby('playerId').LastFiveAvgShotsAgainst.transform(np.max)

summaryGoalieStats=goalieStats[['teamAbbrev','GP','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats['GPRank']=summaryGoalieStats.groupby(['teamAbbrev'])['GP'].rank()
summaryGoalieStats=summaryGoalieStats[['teamAbbrev','GPRank','firstName','lastName','RankAvgShotsAgainst','RankAvgGoalsAgainst','RankAvgSavePctg']].drop_duplicates()

summaryGoalieStats=summaryGoalieStats.rename(columns={"firstName": "GoalieFirstName",'lastName':'GoalieLastName','teamAbbrev':'GoalieteamAbbrev'})
summaryGoalieStats

,GoalieteamAbbrev,GPRank,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,WPG,1.0,Laurent,Brossoit,38.795282,9.309183,79.823083
9,WPG,2.0,Connor,Hellebuyck,35.299073,7.666386,85.004212
35,BOS,1.0,Linus,Ullmark,85.678180,38.668913,91.406908
53,BOS,2.0,Jeremy,Swayman,66.764954,18.491997,96.925021
71,VAN,2.0,Thatcher,Demko,60.067397,26.537489,81.465880
...,...,...,...,...,...,...,...
1060,ANA,1.0,Lukas,Dostal,90.817186,90.480202,21.314238
1075,CHI,2.0,Petr,Mrazek,87.489469,79.064869,44.355518
1100,CHI,1.0,Arvid,Soderblom,71.735468,99.115417,7.708509
1115,SJS,1.0,Kaapo,Kahkonen,95.240101,89.174389,27.506318


In [169]:
full=todaysSkaters.merge(summaryGoalieStats[summaryGoalieStats['GPRank']==1], left_on='Opponent', right_on='GoalieteamAbbrev', how='left').drop(columns=['GoalieteamAbbrev','GPRank'])

full.sort_values(by=['NoOutliersAvgSOG']
                        , ascending=False #or True for players trending down
                       ).reset_index(drop=True).head(25)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,Opponent,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg
0,BOS,8477956,David,Pastrnak,4.948718,5.8,6.2,5.193548,ARI,Karel,Vejmelka,73.083404,77.379949,33.656276
1,TBL,8476453,Nikita,Kucherov,4.341463,3.8,3.6,4.838710,LAK,David,Rittich,2.021904,0.631845,97.893850
2,TOR,8477939,William,Nylander,4.405405,4.0,4.1,4.714286,SJS,Kaapo,Kahkonen,95.240101,89.174389,27.506318
3,TOR,8479318,Auston,Matthews,4.351351,3.6,5.0,4.322581,SJS,Kaapo,Kahkonen,95.240101,89.174389,27.506318
4,OTT,8480801,Brady,Tkachuk,4.542857,5.4,5.0,4.310345,CGY,Dan,Vladar,30.328559,84.161752,14.743050
5,NSH,8476887,Filip,Forsberg,3.850000,5.8,4.1,4.096774,ANA,Lukas,Dostal,90.817186,90.480202,21.314238
6,WPG,8478398,Kyle,Connor,2.615385,0.0,0.0,3.900000,CBJ,Daniil,Tarasov,97.262005,98.188711,32.729570
7,BUF,8475784,Jeff,Skinner,3.275000,2.2,2.6,3.653846,SEA,Chris,Driedger,99.747262,0.084246,100.000000
8,FLA,8477409,Carter,Verhaeghe,3.435897,3.6,3.8,3.625000,STL,Joel,Hofer,27.169334,31.550126,52.021904
9,TOR,8475166,John,Tavares,3.783784,4.8,5.1,3.548387,SJS,Kaapo,Kahkonen,95.240101,89.174389,27.506318


In [172]:
margin=full
margin['lastfivemargin']=margin['LastFiveAvgShots']-margin['NoOutliersAvgSOG']
margin.sort_values(by=['lastfivemargin']
                        , ascending=False #or True for players trending down
                        #, ascending=True
                       ).reset_index(drop=True).head(25)

,teamAbbrev,playerId,firstName,lastName,SeasonAvgSOG,LastFiveAvgShots,LastTenAvgShots,NoOutliersAvgSOG,Opponent,GoalieFirstName,GoalieLastName,RankAvgShotsAgainst,RankAvgGoalsAgainst,RankAvgSavePctg,lastfivemargin
0,VAN,8480012,Elias,Pettersson,2.725000,5.6,3.9,2.666667,NYI,Semyon,Varlamov,84.330244,32.603201,88.037068,2.933333
1,CGY,8476456,Jonathan,Huberdeau,1.700000,3.6,2.2,1.793103,OTT,Anton,Forsberg,21.946083,64.448189,4.759899,1.806897
2,LAK,8479400,Pierre-Luc,Dubois,1.916667,3.6,2.7,1.884615,TBL,Andrei,Vasilevskiy,25.905644,60.867734,36.520640,1.715385
3,NSH,8476887,Filip,Forsberg,3.850000,5.8,4.1,4.096774,ANA,Lukas,Dostal,90.817186,90.480202,21.314238,1.703226
4,CGY,8477496,Elias,Lindholm,2.675000,4.2,3.1,2.685714,OTT,Anton,Forsberg,21.946083,64.448189,4.759899,1.514286
5,WPG,8477940,Nikolaj,Ehlers,2.974359,4.4,3.8,2.942857,CBJ,Daniil,Tarasov,97.262005,98.188711,32.729570,1.457143
6,WPG,8482149,Cole,Perfetti,2.256410,3.6,3.1,2.161290,CBJ,Daniil,Tarasov,97.262005,98.188711,32.729570,1.438710
7,EDM,8477998,Warren,Foegele,2.444444,3.8,3.2,2.370370,CHI,Arvid,Soderblom,71.735468,99.115417,7.708509,1.429630
8,OTT,8480208,Drake,Batherson,2.542857,3.8,3.2,2.451613,CGY,Dan,Vladar,30.328559,84.161752,14.743050,1.348387
9,VAN,8478444,Brock,Boeser,2.775000,4.0,3.3,2.666667,NYI,Semyon,Varlamov,84.330244,32.603201,88.037068,1.333333


In [132]:
# link = BaseURL + '/v1/partner-game/US/now'
# f = requests.get(link)
# f.json()['games'][0]['homeTeam']
# # df=pd.DataFrame(f.json()['games'][0])
# # df